In [2]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()

data_path = "../data/"
q_a_year_path = "../data/qayear/"

### In case of change price while maintain old qty

In [2]:
name = 'MCS'

In [9]:
sql = '''
SELECT B.id, S.name, date, qty, price, qty*price AS amt, status, kind, S.id AS stock_id
FROM buys B
JOIN stocks S ON B.stock_id = S.id 
WHERE name = "%s" AND status = "Active"
ORDER BY date ASC'''
sql = sql % name
buys = pd.read_sql(sql, conpf)
buys

,id,name,date,qty,price,amt,status,kind,stock_id
0,284,MCS,2016-11-11,10000,16.7,167000.0,Active,HD,43
1,740,MCS,2016-11-11,10000,16.7,167000.0,Active,HD,43
2,286,MCS,2016-11-14,10000,16.7,167000.0,Active,HD,43
3,287,MCS,2016-11-14,10000,16.7,167000.0,Active,HD,43
4,288,MCS,2016-11-14,10000,16.7,167000.0,Active,HD,43
5,289,MCS,2016-11-14,10000,16.7,167000.0,Active,HD,43
6,290,MCS,2016-11-17,10000,16.7,167000.0,Active,HD,43
7,653,MCS,2019-11-11,5000,16.7,83500.0,Active,HD,43
8,841,MCS,2021-03-16,5000,16.7,83500.0,Active,HD,43


In [10]:
ttl_qty = buys.qty.sum()
ttl_amt = buys.amt.sum()
unit_cost = ttl_amt / ttl_qty
ttl_amt,ttl_qty,unit_cost

(1336000.0, 80000, 16.7)

In [5]:
sr = buys.id
sr

0    284
1    740
2    286
3    287
4    288
5    289
6    290
7    653
8    841
Name: id, dtype: int64

In [6]:
ids = sr.values
ids

array([284, 740, 286, 287, 288, 289, 290, 653, 841], dtype=int64)

In [7]:
def update_buys_by_id(price,id):
    sql = "UPDATE buys SET price = %s, fee = qty * price * .00207, vat = qty * price * .00207 * .07, net = (qty * price) + fee + vat WHERE id = %s AND status = 'Active'"
    sql = sql % (price,id)
    rp = conpf.execute(sql)
    

In [8]:
for id in ids:
    price = 16.7
    update_buys_by_id(price,id)

### Recalculate cost when unit cost changes

In [4]:
sql = '''
SELECT *
FROM buys B
JOIN stocks S ON B.stock_id = S.id 
WHERE status = "Active" AND name = '%s'
ORDER BY date'''
sql = sql % name
print(sql)


SELECT *
FROM buys B
JOIN stocks S ON B.stock_id = S.id 
WHERE status = "Active" AND name = 'TTLPF'
ORDER BY date


In [5]:
df_tmp = pd.read_sql(sql, conpf)
df_tmp

,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart,id,name,category_id,website
0,716,154,2020-05-20,1000,21.9,45.33,3.17,Active,21948.50,DIV,None,154,TTLPF,3,
1,717,154,2020-05-21,1000,22.0,45.54,3.19,Active,22048.73,DIV,None,154,TTLPF,3,
2,718,154,2020-05-22,1000,21.9,45.33,3.17,Active,21948.50,DIV,None,154,TTLPF,3,
3,724,154,2020-05-29,500,21.5,22.25,1.56,Active,10773.81,DIV,None,154,TTLPF,3,
4,725,154,2020-06-01,300,21.5,13.35,0.93,Active,6464.28,DIV,None,154,TTLPF,3,
5,728,154,2020-06-09,300,21.4,13.29,0.93,Active,6434.22,DIV,None,154,TTLPF,3,
6,744,154,2020-06-23,700,21.4,31.01,2.17,Active,15013.18,DIV,None,154,TTLPF,3,
7,748,154,2020-07-17,500,21.5,22.25,1.56,Active,10773.81,DIV,None,154,TTLPF,3,
8,755,154,2020-07-31,400,21.5,17.80,1.25,Active,8619.05,DIV,None,154,TTLPF,3,
9,786,154,2020-11-10,100,21.1,4.37,0.31,Active,2114.68,DIV,None,154,TTLPF,3,


In [6]:
def update_buys(qty,price,id):
    sql = "UPDATE buys SET qty = %s, price = %s, fee = qty * price * .00207, vat = qty * price * .00207 * .07, net = (qty * price) + fee + vat WHERE id = %s AND status = 'Active'"
    sql = sql % (qty,price,id)
    rp = conpf.execute(sql)
    return "Records updated = " + str(rp.rowcount)

In [7]:
id = 724
qty = 500
price = 21.7
fee = round(qty * price * .00207,2)
vat = round(qty * price * .00207 * .07,2)
net = (qty * price) + fee + vat
print(id,qty,price,fee,vat,net)

724 500 21.7 22.46 1.57 10874.029999999999


In [8]:
update_buys(qty,price,id)

'Records updated = 1'

### Delete records en masse

In [6]:
name = 'TTLPF'
sql = '''
SELECT B.id, S.name, date, qty, price, qty*price AS amt, status, kind, S.id AS stock_id
FROM buys B
JOIN stocks S ON B.stock_id = S.id 
WHERE name = "%s" AND status = "Active"
AND B.id != 716
ORDER BY date ASC'''
sql = sql % name
buys = pd.read_sql(sql, conpf)
buys

,id,name,date,qty,price,amt,status,kind,stock_id
0,717,TTLPF,2020-05-21,1000,21.7,21700.0,Active,DIV,154
1,718,TTLPF,2020-05-22,1000,21.7,21700.0,Active,DIV,154
2,725,TTLPF,2020-06-01,300,21.7,6510.0,Active,DIV,154
3,728,TTLPF,2020-06-09,300,21.7,6510.0,Active,DIV,154
4,744,TTLPF,2020-06-23,700,21.7,15190.0,Active,DIV,154
5,748,TTLPF,2020-07-17,500,21.7,10850.0,Active,DIV,154
6,755,TTLPF,2020-07-31,400,21.7,8680.0,Active,DIV,154
7,786,TTLPF,2020-11-10,100,21.7,2170.0,Active,DIV,154


In [7]:
sr = buys.id
sr

0    717
1    718
2    725
3    728
4    744
5    748
6    755
7    786
Name: id, dtype: int64

In [14]:
ids = sr.values
ids

array([717, 718, 725, 728, 744, 748, 755, 786], dtype=int64)

In [17]:
def delete_buys_by_id(id):
    sql = "DELETE FROM buys WHERE id = %s AND status = 'Active'"
    sql = sql % (id)
    rp = conpf.execute(sql)

In [15]:
sql = '''
DELETE FROM buys WHERE id IN %s AND status = 'Active'
'''
sql = sql % (ids)
print(sql)


DELETE FROM buys WHERE id IN [717 718 725 728 744 748 755 786] AND status = 'Active'



In [18]:
for id in ids:
    delete_buys_by_id(id)

### Summarized profit by name

In [8]:
sql = '''
SELECT name, sum(profit) AS profit
FROM sells JOIN buys ON sells.buy_id = buys.id
JOIN stocks ON buys.stock_id = stocks.id
GROUP BY stocks.name
ORDER BY sum(profit) DESC LIMIT 10'''
profits_by_stock = pd.read_sql(sql, conpf)
profits_by_stock

,name,profit
0,CPNREIT,545770.06
1,THANI,237046.32
2,BEAUTY,192974.08
3,JMT,157072.41
4,MTC,155475.08
5,STA,130902.48
6,SAWAD,115506.59
7,SENA,90977.08
8,KTC,89973.84
9,CPALL,87645.62


In [9]:
sql = '''
SELECT YEAR(date) AS year, MONTH(date) AS month, profit
FROM sells 
'''
sells = pd.read_sql(sql, conpf)
sells.dtypes

year        int64
month       int64
profit    float64
dtype: object

In [10]:
grouped = sells.groupby(['year','month'])
grouped

In [11]:
profit_by_month = grouped['profit'].sum()
profit_by_month

year  month
2016  2        -35100.86
2017  1         -7824.90
      2         39018.75
      5        -29169.21
      6         23019.69
      7         38801.41
      8         44544.74
      9        101089.25
      10      -286213.31
      11       159183.48
      12       145236.90
2018  1        185200.08
      2        270867.16
      3       -103935.40
      4        -74532.83
      5       -174556.37
      6        -42911.69
      7         62005.09
      8        -18978.90
      9         37425.26
      10        27009.95
      11        30595.44
      12       -38431.94
2019  1         47490.10
      2        -85974.92
      3          1694.48
      4        -18481.13
      5         41561.91
      6         50055.90
      7       -103429.59
      8        129268.71
      9         -6309.57
      10        75913.61
      11       -90451.97
      12       -30920.31
2020  1       -182804.13
      3       -147177.39
      4        -37432.77
      5       -389834.16
      6      

In [12]:
grouped.agg(['sum','mean', 'max', 'min','count'])

profit                                          
                  sum          mean        max        min count
year month                                                     
2016 2      -35100.86 -35100.860000  -35100.86  -35100.86     1
2017 1       -7824.90  -7824.900000   -7824.90   -7824.90     1
     2       39018.75  13006.250000   22941.28    4328.89     3
     5      -29169.21  -9723.070000    9470.07  -19444.36     3
     6       23019.69   3288.527143   18199.05  -29317.44     7
     7       38801.41  12933.803333   17012.15    6566.42     3
     8       44544.74  11136.185000   32950.14   -5675.55     4
     9      101089.25  25272.312500   38439.63   11106.58     4
     10    -286213.31 -20443.807857   19837.21 -415882.22    14
     11     159183.48   8378.077895   20605.96  -40753.07    19
     12     145236.90   9682.460000   16590.25    2228.12    15
2018 1      185200.08   8418.185455   17040.40  -43147.32    22
     2      270867.16  14256.166316  155079.49  -14446.55    19
     3     -103935.40 -34645.133333    1299.44  -97892.61     3
     4      -74532.83 -24844.276667   11980.39  -97892.61     3
     5     -174556.37 -13427.413077   18488.36 -106096.38    13
     6      -42911.69  -6130.241429   10015.50  -37020.63     7
     7       62005.09   7750.636250   32736.35  -30092.71     8
     8      -18978.90  -2108.766667   15752.46  -19893.25     9
     9       37425.26   2495.017333   21397.41  -51272.44    15
     10      27009.95  27009.950000   27009.95   27009.95     1
     11      30595.44   7648.860000   15918.59  -12670.30     4
     12     -38431.94 -38431.940000  -38431.94  -38431.94     1
2019 1       47490.10  23745.050000   30194.31   17295.79     2
     2      -85974.92 -14329.153333   34405.27  -75067.85     6
     3        1694.48    564.826667    1398.63    -653.45     3
     4      -18481.13  -9240.565000    4378.69  -22859.82     2
     5       41561.91  20780.955000   40827.75     734.16     2
     6       50055.90   7150.842857   24312.72  -30899.09     7
     7     -103429.59 -25857.397500   47463.94  -61798.19     4
     8      129268.71  32317.177500   49451.81   17492.74     4
     9       -6309.57  -3154.785000   56935.21  -63244.78     2
     10      75913.61  75913.610000   75913.61   75913.61     1
     11     -90451.97 -30150.656667  -29219.39  -31228.62     3
     12     -30920.31  -5153.385000   31418.20  -41791.48     6
2020 1     -182804.13 -60934.710000  -53913.30  -74079.52     3
     3     -147177.39 -18397.173750  194977.80 -213635.68     8
     4      -37432.77  -2079.598333   20328.89  -30261.36    18
     5     -389834.16 -25988.944000   14488.36  -74655.61    15
     6     -241008.23 -15063.014375   26586.35  -58442.98    16
     7     -204522.16 -34087.026667   15021.02  -75653.40     6
     8      -93407.30 -10378.588889   19871.52 -152425.93     9
     9      -63826.53  -7978.316250   12074.71  -17934.57     8
     10      86694.25  12384.892857   13106.31   10511.58     7
     12    -112227.33 -28056.832500   16195.47  -50789.88     4
2021 1       51296.67   8549.445000   26592.67  -38446.55     6
     2      202547.56  20254.756000   40643.41     236.43    10
     3      -19735.49   -704.838929   21118.92  -14043.21    28
     4       65267.16   4661.940000   16041.52   -7426.10    14
     5       71789.63   3418.553810   28913.18  -22633.46    21